In [2]:
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4


## Load heatwaves and get initial control dates

In [3]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/france/THI_france_90_2022.csv')

In [4]:
heatwaves.month.unique()

array([6, 7, 8, 5, 9])

In [5]:
#may_hw = heatwaves.groupby('heat_ID').apply(lambda x: 5 in x['month'].unique()).reset_index()
#may_hw = may_hw[may_hw[0] == True]
#heatwaves = heatwaves[~heatwaves['heat_ID'].isin(may_hw['heat_ID'].values)]

In [6]:
heatwaves['month'].unique()

array([6, 7, 8, 5, 9])

In [7]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [8]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)


In [9]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [10]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [11]:
heat_days = heat_days.to_dict()['datetime']

In [12]:
heatwaves_active.reset_index(inplace=True)

In [13]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/france/daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df = full_df[~full_df['month'].isin([4, 10])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]


In [14]:
full_df

,Unnamed: 0,datetime,number,spatial_ref,t2m,d2m,rh,thi,adm2,adm1,day,month,year,location,datetime_ord
0,0,2018-05-01,0.0,0.0,12.984543,3.212509,72.96763,11.467585,Bas-Rhin,Alsace,1,5,2018,"Bas-Rhin, Alsace",736815
1,1,2018-05-02,0.0,0.0,16.846756,8.483494,82.40495,15.875832,Bas-Rhin,Alsace,2,5,2018,"Bas-Rhin, Alsace",736816
2,2,2018-05-03,0.0,0.0,16.842527,7.163579,81.98801,15.847073,Bas-Rhin,Alsace,3,5,2018,"Bas-Rhin, Alsace",736817
3,3,2018-05-04,0.0,0.0,20.003017,9.834668,72.43605,19.402456,Bas-Rhin,Alsace,4,5,2018,"Bas-Rhin, Alsace",736818
4,4,2018-05-05,0.0,0.0,22.701967,11.183942,65.62170,22.267851,Bas-Rhin,Alsace,5,5,2018,"Bas-Rhin, Alsace",736819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74393,74393,2022-09-26,0.0,0.0,9.025503,3.669450,98.37174,7.805842,Savoie,Rhône-Alpes,26,9,2022,"Savoie, Rhône-Alpes",738424
74394,74394,2022-09-27,0.0,0.0,5.570193,2.507942,96.72386,4.936991,Savoie,Rhône-Alpes,27,9,2022,"Savoie, Rhône-Alpes",738425
74395,74395,2022-09-28,0.0,0.0,7.894247,6.730359,97.35292,7.502293,Savoie,Rhône-Alpes,28,9,2022,"Savoie, Rhône-Alpes",738426
74396,74396,2022-09-29,0.0,0.0,6.146764,4.305479,97.17700,5.682936,Savoie,Rhône-Alpes,29,9,2022,"Savoie, Rhône-Alpes",738427


In [15]:
full_df['datetime'] = pd.to_datetime(full_df[['day', 'month', 'year']])
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]



In [16]:
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']

In [17]:
full_df.drop_duplicates(['location', 'datetime'], inplace=True)



In [19]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [20]:
full_df['temp_pct'].describe()

count    73440.000000
mean         0.500654
std          0.288677
min          0.001307
25%          0.250980
50%          0.500654
75%          0.750327
max          1.000000
Name: temp_pct, dtype: float64

In [21]:
heatwaves_active['num_iters'] = 'test'

In [24]:

for index, row in heatwaves_active.iterrows():
    
    i = 0
    try:
        while ((full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]>= 0.8) | 
               (full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]<= 0.2)):


                heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        
        while ((full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]>= 0.8) | 
               (full_df[(full_df['datetime'] == heatwaves_active.iloc[index, -3]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            
            
    print(index / len(heatwaves_active))
            
    heatwaves_active.iloc[index, -1] = i

0.0
0.0005241090146750524
0.0010482180293501049
0.0015723270440251573
0.0020964360587002098
0.002620545073375262
0.0031446540880503146
0.003668763102725367
0.0041928721174004195
0.0047169811320754715
0.005241090146750524
0.005765199161425576
0.006289308176100629
0.006813417190775681
0.007337526205450734
0.007861635220125786
0.008385744234800839
0.00890985324947589
0.009433962264150943
0.009958071278825996
0.010482180293501049
0.0110062893081761
0.011530398322851153
0.012054507337526206
0.012578616352201259
0.01310272536687631
0.013626834381551363
0.014150943396226415
0.014675052410901468
0.01519916142557652
0.015723270440251572
0.016247379454926623
0.016771488469601678
0.01729559748427673
0.01781970649895178
0.018343815513626835
0.018867924528301886
0.01939203354297694
0.019916142557651992
0.020440251572327043
0.020964360587002098
0.02148846960167715
0.0220125786163522
0.022536687631027254
0.023060796645702306
0.02358490566037736
0.02410901467505241
0.024633123689727462
0.0251572327044

In [25]:
heatwaves_active['control_day'].min()

Timestamp('2022-05-14 00:00:00')

In [26]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves_active.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/france/THI_heatwave_control_90.csv')